# Regresión Logística - Creando una red neuronal simple

Este documento implementará el código necesario para construir la arquitectura general de un algoritmo de aprendizaje y finalmente unir las funciones en un modelo principal.
Los datos que utilizaremos en esta libreta se han obtenido de la plataforma Coursera y además los puedes encontrar en mi repositorio. 

### El objetivo de este documento es crear un modelo que permita identificar si una imagen es de un gato o no lo es.

In [1]:
#Vamos a importar todos los paquetes que sean necesarios para trabajar en este notebook
import numpy as np
import matplotlib.pyplot as plt
import h5py #para cargar los sets de datos que vienen en formato H5
import scipy

In [2]:
#Definimos una función load_dataset que cargue el set de entrenamiento y el set e testeo
def load_dataset():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # características del set de entrenamiento
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # etiqueta (gato o no gato) 

    test_dataset = h5py.File('test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # características del set de testeo
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # etiqueta (gato o no gato)

    classes = np.array(test_dataset["list_classes"][:]) 
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes



El set de entrenamiento tiene un total de "m_train" de imágenes etiquetadas como gato (y=1) o no gato (y=0). Para el caso del set de testeo o prueba tenemos "m_test" imágenes. Las dimensiones de las imágenes serán (num_px, num_px, 3), donde num_px es el número de píxeles en altura y anchura, y el 3 es para los canales RGB.

In [3]:
#Como vamos a pre-procesar estos sets, se añade un _orig al final para diferenciarlos de los que ya
#han sido pre-procesados.
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset() 

In [4]:
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

In [5]:
#Veamos las dimensones de las X de entrenamiento y testeo:
print('Dimensiones X entrenamiento: ' + str(train_set_x_orig.shape) )
print('Dimensiones X testeo: '+ str(test_set_x_orig.shape))

Dimensiones X entrenamiento: (209, 64, 64, 3)
Dimensiones X testeo: (50, 64, 64, 3)


Para trabajar debemos convertir las matrices en arrays de dimensión (num_px * num_px * 3, 1). Es decir, un vector. Para esto utilizamos reshape.

In [6]:
train_set_x_vector = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_vector = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T

Vamos a estandarizar nuestros datos dividiendo cada fila del set de datos entre 255, que es el valor máximo de un canal de píxeles.

In [7]:
x_train = train_set_x_vector/255
x_test = test_set_x_vector/255

Ahora debemos diseñar un algoritmo que tenga de entrada x_train, implemente la función con los parámetros w y b, aplique la función sigmoide (que vamos a definir a continuación) y finalmente comience la optimización de la función de coste.


Para cada $x^{(i)}$:
$$z^{(i)} = w^T x^{(i)} + b $$
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})$$ 
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})$$

Calculamos la función de coste de la siguiente forma:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})$$

In [8]:
#Definimos la función sigmoide
def sigmoid(z):
    s = 1/ ( 1 + np.exp(-z))
    return s

El procedimiento que vamos a seguir es el siguiente:
Iniciaremos los parámetros w y b con valor cero. Comenzarán entonces las iteraciones:
1) Calcular las pérdidas (propagación hacia delante),
2) Calcular el gradiente (propagación hacia atrás) y
3) Actualizar parámetros (descenso de gradiente).

In [9]:
#Iniciar los parámetros con ceros
def ceros(dims):
    """
    Esta función creará vectores w nulos de dimensión (dims, 1) y hará que b sea cero
    dims == tamaño de w
    """
    w = np.zeros((dims,1))
    b = 0
    return w,b

Hasta ahora tenemos X y los parámetros W, b. A continuación debemos implementar la función propagación que permita computar la función de coste y su gradiente.
- Tenemos X
- Calculamos el vector $A = \sigma(w^T X + b) = (a^{(1)}, a^{(2)}, ..., a^{(m-1)}, a^{(m)})$
- Y la función de coste viene dada por  $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$

Las derivadas que necesitaremos son las siguientes:
$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})$$

In [10]:
def propagacion(w,b,X,Y):
    m= X.shape[1]
    
    A= sigmoid(np.dot (w.T,X) + b) #Calculamos la activación
    cost = -1/m * np.sum(Y*np.log(A) + (1-Y)*np.log(1-A)  ) #Calculamos la función de coste
    
    dw = 1/m*np.dot(X,(A-Y).T)
    db = 1/m*np.sum(A-Y)
    
    #Utilizamos np.squeeze para eliminar dimensiones en cost
    cost = np.squeeze(cost)
    return dw , db, cost

Comienza la optimización. Ya tenemos los paráemtros iniciados y hemos definido la función de propagación para calcular la activación, la función de coste y su gradiente. Ahora, debemos ser capaces de actualizar los parámetros a partir del descenso de gradiente.

In [11]:
def optimizar(w, b, X, Y, iters, alpha):
    """
    X- datos de entrada
    Y- vector de salida con resultados REALES (no probables, sino los de verdad!!)
    iters - número de iteraciones
    alpha- tasa de aprendizaje (learning rate)

"""
    for i in range(iters):
        
        dw, db, cost = propagacion(w,b,X,Y)
    
        w = w - alpha*dw
        b = b - alpha*db
        
    parametros = {"w": w, "b": b }
    gradientes = {"dw": dw, "db": db}
    return parametros, gradientes

Ahora que ya tenemos el algoritmo que optimiza los parámetros, queda definir una función que sea capaz de hacer las predicciones. Esto es, calcular $\hat{Y}$ y además convertir los resultados en 0 o en 1 según la activación sea mayor o menor que 0.5; para guardar las predicciones crearemos un vector llamado "Y_preds"

In [12]:
def predecir(w,b,X):
    m= X.shape[1]
    Y_preds = np.zeros((1,m)) #Creamos el vector de predicciones con ceros
    w = w.reshape(X.shape[0],1)
    
    A= sigmoid(np.dot(w.T,X) + b)
    
    for i in range(A.shape[1]):
        if A[0][i] >0.5:
            Y_preds[0][i]=1
        else:
            Y_preds[0][i]=0
    return Y_preds

# De momento...

Hemos iniciado los parámetros w y b. Luego, optimizamos de forma iterativa para obtener los parámetros w y b. Por último se realizaron predicciones utilizando los parámetros aprendidos.

Ahora queda implementarlo todo en un modelo.

In [17]:
def modelo_final(X_train, Y_train, X_test, Y_test, iterations, alpha):
    w, b = ceros(X_train.shape[0]) #Iniciar los parámetros en cero
    parametros,gradientes = optimizar(w,b, X_train, Y_train, iterations, alpha) #Optimizar
    w = parametros['w'] #Cogemos el w aprendido
    b = parametros['b'] #Cogemos el b aprendido
    
    #Vamos a hacer las predicciones en los sets de entrenamiento y testeo
    Y_pred_train = predecir(w,b,X_train)
    Y_pred_test = predecir(w,b,X_test)
    
    print("Precisión entrenamiento: {} %".format(100 - np.mean(np.abs(Y_pred_train - Y_train)) * 100))
    print("Precisión test: {} %".format(100 - np.mean(np.abs(Y_pred_test - Y_test)) * 100))
    
    final = {"w": w, "b": b, "Predicción Y entrenamiento":Y_pred_train, "Predicción Y testeo":Y_pred_test}
    return final

In [26]:
final = modelo_final(x_train, train_set_y, x_test, test_set_y, iterations= 1800, alpha= 0.005 )

Precisión entrenamiento: 98.56459330143541 %
Precisión test: 70.0 %


Como vemos, nuestro modelo tiene una precisión con el set de entrenamiento de un 98%. Para el caso del set de testeo tenemos un 70%.